# SQL RAG with SQLite + Vector Search

This lab mirrors the Snowflake Cortex RAG demo using SQLite for storage and NumPy for similarity.

**Learning objectives**
- Store documents + embeddings in SQLite
- Perform semantic search with cosine similarity
- Build a simple RAG pipeline


In [ ]:
# Optional: install deps if running in a fresh environment
# !pip install openai python-dotenv numpy


In [ ]:
import os
import json
import sqlite3
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))


In [ ]:
conn = sqlite3.connect('knowledge_base.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS documents (
        doc_id INTEGER PRIMARY KEY,
        content TEXT,
        embedding TEXT
    )
''')

documents = [
    'Employees are allowed to work remotely up to two days per week.',
    'Annual leave requests should be submitted at least two weeks in advance.',
    'The company provides full health insurance coverage to all permanent employees.',
    'IT tickets are answered within 24 hours during business days.',
    'Expense reports must be filed within 30 days of purchase.'
]


In [ ]:
def get_embedding(text):
    response = client.embeddings.create(
        model='text-embedding-3-small',
        input=text
    )
    return response.data[0].embedding

cursor.execute('DELETE FROM documents')
for i, doc in enumerate(documents, start=1):
    embedding = get_embedding(doc)
    cursor.execute(
        'INSERT INTO documents (doc_id, content, embedding) VALUES (?, ?, ?)',
        (i, doc, json.dumps(embedding))
    )
conn.commit()


In [ ]:
# SQL keyword query (traditional search)
cursor.execute("SELECT doc_id, content FROM documents WHERE content LIKE '%remote%'")
cursor.fetchall()


In [ ]:
def cosine_similarity(a, b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

def search_documents(query, top_k=3):
    query_embedding = get_embedding(query)
    cursor.execute('SELECT doc_id, content, embedding FROM documents')
    results = []
    for doc_id, content, emb_json in cursor.fetchall():
        emb = json.loads(emb_json)
        similarity = cosine_similarity(query_embedding, emb)
        results.append((doc_id, content, similarity))
    results.sort(key=lambda x: x[2], reverse=True)
    return results[:top_k]


In [ ]:
def rag_query(question):
    relevant_docs = search_documents(question)
    context = '\n'.join([doc[1] for doc in relevant_docs])
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {'role': 'system', 'content': f'Answer based on this context:\n{context}'},
            {'role': 'user', 'content': question}
        ]
    )
    return response.choices[0].message.content.strip(), relevant_docs

answer, sources = rag_query('Does the company allow remote work?')
answer, [s[1] for s in sources]


## Snowflake Cortex mapping

| Snowflake Cortex | SQLite + OpenAI equivalent |
|---|---|
| `SNOWFLAKE.CORTEX.EMBED_TEXT_768(model, text)` | `client.embeddings.create(...)` |
| `VECTOR_COSINE_SIMILARITY(v1, v2)` | `np.dot(a, b) / (norm(a) * norm(b))` |
| `SNOWFLAKE.CORTEX.COMPLETE(model, prompt)` | `client.chat.completions.create(...)` |
